In [5]:
import pandas as pd
import numpy as np
import xlsxwriter

import openpyxl
import mysql.connector

In [6]:
# Connecting to the server
def open_conn():
    conn = mysql.connector.connect(user='root', password='vini1985',
                              host='127.0.0.1', database='ulb_data',
                              auth_plugin='mysql_native_password')
    print("DB Connection Opened!")
    return conn
 
# Disconnecting from the server
def close_con(conn):
    conn.close()
    print("DB connection closed")

In [7]:
query = "select ulb_hrms_id, ulb_name from ulbs where ulb_name like '%robert%'"

In [8]:
conn = open_conn()
mycursor = conn.cursor()
mycursor.execute(query)
myresult = mycursor.fetchall()
for x in myresult:
  print(x)

close_con(conn)

InterfaceError: 2003: Can't connect to MySQL server on '127.0.0.1:3306' (61 Connection refused)

In [ ]:
path = '.\Budget_Files_2024-25'

In [11]:
# load excel with its path
wrkbk = openpyxl.load_workbook("Budget_Files_2024-25/Appendix-B_2024-25_For_CCs.xlsx", data_only=True)
  
sh = wrkbk.active

# sh.cell(row=4,column=4).value
# sh.cell(row=4,column=24).value

# ulb_name = sh['D4'].value
# ulb_hrms_code = sh['X4'].value

# iterate through excel and display data
for i in range(9, 3008):
    if sh.cell(row=i, column=2).value == None:
        break
    else:
        for j in range(2, 26):
            if j==25:
                print(sh.cell(row=i, column=j).value)
                break
            else:
                print(sh.cell(row=i, column=j).value, end=",")

# print(ulb_name, ulb_hrms_code)

911254678,OPS,None,sdfsdfsdfd,F,PK,17000,28950,17000,204000,2024-07-01 00:00:00,400,3600,207600,91344,16608,2400,8400,4800,PP,1200,13148,-,345500
911257426,NPS,110145623852,dsdfsdfsd,M,CAUO,74400,109600,74400,892800,2025-01-01 00:00:00,1900,5700,898500,395340,71880,0,0,6000,SFN,1500,56905,181137.6,1611262.6
911121651,NPS,123456789123,sxdfcsdfd,F,RO,43100,83900,43100,517200,2025-01-01 00:00:00,1100,3300,520500,229020,41640,0,0,3434,SFN,1500,32965,104932.8,933991.8


In [12]:
df = pd.DataFrame(columns=['ulb_name', 'ulb_hrms_code', 'emp_id', 'pension_type', 'pran', 'emp_name', 'gender', 'designation',
                           'payscale_min', 'payscale_max', 'basic_pay', 'basic_pay_for_year', 'increment_month', 'increment_rate',
                           'increment_for_year', 'basic_incrnt_total', 'da_for_year', 'hra_for_year', 'ma_for_year',
                           'ua_pk_spcl_for_year','cca_for_year', 'other_allowance_type', 'oa_amount_for_year',
                           'surrender_leave_encash','nps_contr_for_year', 'total'])
df

,ulb_name,ulb_hrms_code,emp_id,pension_type,pran,emp_name,gender,designation,payscale_min,payscale_max,...,da_for_year,hra_for_year,ma_for_year,ua_pk_spcl_for_year,cca_for_year,other_allowance_type,oa_amount_for_year,surrender_leave_encash,nps_contr_for_year,total


In [13]:
for i in range(9, 3008):
    list.insert(0,sh.cell(4,4).value)
    list.insert(1,sh.cell(4,24).value)
    if sh.cell(row=i, column=2).value == None:
        count = i - 10
        break
    else:
        for j in range(2, 26):
            if j==25:
                cell_obj = sh.cell(row=i, column=j).value
                break
            else:
                cell_obj = sh.cell(row=i, column=j).value
                if type(cell_obj) == 'int':
                    list.insert(j, cell_obj)
                elif type(cell_obj) == 'str':
                    list.insert(j,"'" + cell_obj + "'")
                else:
                    list.insert(j, cell_obj)
                for x in range(count):
                    df.iloc[count] = list
                    list=[]

TypeError: descriptor 'insert' for 'list' objects doesn't apply to a 'int' object

In [ ]:
list